기존 gpt_tag_weight 가 있는 csv파일을  openai api에서 제공하는 학습 데이터 형태의jsonl로 변환하는 코드

In [ ]:
!pip install openai

In [3]:
import os
import openai
import json
import csv
import pandas as pd
from openai import OpenAI

f = open("chatgptkey.txt", 'r')
line = f.readline()
openai.api_key = line

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key
)

gpt_tag_weight 파일을 가져옴

In [4]:
df1 = pd.read_csv('/content/drive/MyDrive/colab_github/sooljari-crawling/gpt/gpt_tag_weight_refined.csv')
frame=pd.DataFrame(df1)

In [11]:
df1

,Unnamed: 0.1,Unnamed: 0,술 이름,텍스트 내용,tags,weight
0,0,0,고도리 복숭아 와인,GAP인증 복숭아로 만든 디저트 와인\n복숭아의 당도가 가장 높은 8월 말에 수확하...,"['와인', '복숭아', '디저트', '당도', '발효', '맛', '향', '냉동...","[8, 9, 6, 7, 7, 8, 8, 5, 7, 6, 8, 6, 6, 6]"
1,1,1,고도리 화이트,GAP인증 거봉 포도로 만든 신의 물방울\n포도의 당도가 가장 높은 10월 말에 수...,"['와인', '화이트와인', '과일향', '아로마틱', '달콤함', '산미', '고...","[8, 7, 7, 6, 6, 6, 5, 5, 4]"
2,2,2,매실원주,황매실로 담근 100% 매실주\n\n국내 유일 기타 과실주를 첨가하지 않고 매실주 ...,"['매실주', '황매실', '100%', '제주도꿀', '전통제조방법', '향미',...","[10, 9, 5, 7, 8, 6, 9]"
3,3,3,엠,화사한 메밀향과 스파이시한 목넘김\n반전매력 가득 메밀 증류주\n\n프레시하고 화사...,"['메밀향', '스파이시한', '반전매력', '프레시', '화사한', '정통', '...","[9, 8, 9, 7, 7, 8, 7, 6, 8]"
4,4,4,술예쁘다,홍국쌀 특유의 과일향과 새콤달콤함이 인상적인 삼양주 탁주,"['삼양주', '탁주', '과일향', '새콤달콤함']","[10, 9, 8, 7]"
...,...,...,...,...,...,...
143,143,143,매화수,‘저온 냉동 여과공법’을 사용해 부드럽고 깨끗한 맛!\n매실의 풍미와 달콤함을 부담...,"['부드럽고깨끗한맛', '매실의풍미', '달콤함', '12도도수', '매화꽃디자인'...","[8, 9, 7, 6, 5, 8, 7, 7, 6, 7, 8, 6, 6, 6, 6, ..."
144,144,144,15년 숙성 매취순,‘매취순’은 국내 최대 매실농원인 전남 해남 ‘보해매실농원’에서 직접 수확한 국산 ...,"['매취순', '매실주', '청매실', '프리미엄', '국산', '해남', '보해매...","[9, 8, 7, 6, 5, 4, 3, 2, 2, 1]"
145,145,145,대윤 가야곡 왕주,'대윤가야곡주조'는 오랜 전통의 '가야곡왕주'가 '대윤가야곡주조'로 새출발을 하며...,"['대윤가야곡주조', '가야곡왕주', '약주', '궁중술', '왕주', '역사', ...","[10, 9, 8, 7, 7, 6, 6, 5, 5, 4, 4, 3, 3, 2, 2,..."
146,146,146,가야의 魂,"가야 양조장의 대표 막걸리\n\n 인공 감미료, 첨가물을 넣지 않고 ...","['대표막걸리', '인공감미료없음', '첨가물없음', '가야양조장', '발효기술',...","[8, 9, 9, 7, 8, 8, 9, 8, 9]"


가져온 데이터의 무결성을 검사하고 line별로 json로 변환함 (JSONLine)



In [ ]:
pip install tiktoken

In [25]:
gpt_tag_errorline=[]
jsonlines=[]

In [26]:
import ast
from functools import reduce

for idx,row in df1.iterrows():
  if len(row['tags'].split(','))!=len(row['weight'].split(',')):
    gpt_tag_errorline.append(idx)
    continue
  description=row['텍스트 내용']
  taglst=ast.literal_eval(row['tags'])
  weightlst=ast.literal_eval(row['weight'])
  tagtostr=",".join(taglst)
  add = lambda x, y: str(x) + "," + str(y)
  weighttostr = reduce(add, weightlst)
  to_json={"messages": [{"role": "system", "content": "주류 데이터를 검색하기 위한 태그를 설명문으로부터 추출하고 싶어. 술에 대한 정보를 포함하는 10자 이내의 태그와 각 태그의 중요도를 ,로 묶어서 출력하고 싶어. "},
{"role": "system", "content": "또한 추출한 태그의 중요도를 1에서 10 사이의 수치로 표현해줘. 각 태그가 주관적인지 객관적인지 판단해서, 맛이나 향 등을 포함하거나 주관적이라면 높은 점수를 줘."},
{"role": "user", "content": description},{"role": "assistant", "content":tagtostr+"\n"+weighttostr}]}
  jsonlines.append(to_json)

In [27]:

from collections import OrderedDict
with open("target_data.jsonl", "w", encoding="utf-8") as f:
  for datas in jsonlines:
    json.dump(datas, f, ensure_ascii=False) # ensure_ascii로 한글이 깨지지 않게 저장
    f.write("\n") # json을 쓰는 것과 같지만, 여러 줄을 써주는 것이므로 "\n"을 붙여준다.
  f.close()

변환해서 만든 데이터가 학습에 적합한지 확인하는 openai의 코드

In [30]:
data_path = "target_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 140
First example:
{'role': 'system', 'content': '주류 데이터를 검색하기 위한 태그를 설명문으로부터 추출하고 싶어. 술에 대한 정보를 포함하는 10자 이내의 태그와 각 태그의 중요도를 ,로 묶어서 출력하고 싶어. '}
{'role': 'system', 'content': '또한 추출한 태그의 중요도를 1에서 10 사이의 수치로 표현해줘. 각 태그가 주관적인지 객관적인지 판단해서, 맛이나 향 등을 포함하거나 주관적이라면 높은 점수를 줘.'}
{'role': 'user', 'content': 'GAP인증 복숭아로 만든 디저트 와인\n복숭아의 당도가 가장 높은 8월 말에 수확하여 별도의 당을 첨가하지 안히고 순수 발효한 와인이며 복숭아를 발효하여 끝에서 느껴지는 복숭아의 부드럽고 향긋한 맛과 향이 특징입니다\n고도리 복숭아와인은 복숭아를 8월 말 가장 당이 높고 향기가 무르익었을 때 수확하여 -10도에서 20일간 냉동 숙성시켜, 당이 35BX되었을때 파쇄 후 발효시켜 만든 복숭아 아이스 와인입니다. 풍부한 복숭아 향을 담고 있어 더위의 갈증을 시원하게 해소시켜 줍니다.\n고도리 복숭아와인은 미디엄 스위트 와인으로 풍부한 산과 복숭아의 아로마 향으로 생선 요리나 흰 살코기 닭과 오리를 이용한 요리와 잘 어울립니다.'}
{'role': 'assistant', 'content': '와인,복숭아,디저트,당도,발효,맛,향,냉동숙성,아이스와인,산,아로마향,생선요리,살코기닭,오리\n8,9,6,7,7,8,8,5,7,6,8,6,6,6'}


In [31]:
# Format error checks
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


유용한 함수들 정의 가져옴

In [33]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

오류 체크 및 예상 토큰 개수 계산

In [34]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 4, 4
mean / median: 4.0, 4.0
p5 / p95: 4.0, 4.0

#### Distribution of num_total_tokens_per_example:
min / max: 231, 1336
mean / median: 467.8071428571429, 435.5
p5 / p95: 262.0, 703.1000000000004

#### Distribution of num_assistant_tokens_per_example:
min / max: 18, 152
mean / median: 65.87857142857143, 68.5
p5 / p95: 29.0, 99.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


계산한 토큰 수 및 학습 시간으로 추측할 수 있는 가격 계산

In [35]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~65493 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~196479 tokens


학습 시작

In [36]:
client.files.create(
  file=open("target_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-AtyWWLfHIXPKLCX9SgvZw0eO', bytes=169457, created_at=1702107280, filename='target_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [37]:
client.fine_tuning.jobs.create(
  training_file="file-AtyWWLfHIXPKLCX9SgvZw0eO",
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-BVJ1pyw1eUaykUhCqY6xR5VP', created_at=1702107506, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-eOujTwd2bK4aMqI7Oe49Tygg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-AtyWWLfHIXPKLCX9SgvZw0eO', validation_file=None)

In [38]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-BVJ1pyw1eUaykUhCqY6xR5VP', created_at=1702107506, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-eOujTwd2bK4aMqI7Oe49Tygg', result_files=[], status='running', trained_tokens=None, training_file='file-AtyWWLfHIXPKLCX9SgvZw0eO', validation_file=None)], object='list', has_more=False)

In [40]:
modelid='ftjob-BVJ1pyw1eUaykUhCqY6xR5VP'

In [43]:
client.fine_tuning.jobs.retrieve(modelid)

FineTuningJob(id='ftjob-BVJ1pyw1eUaykUhCqY6xR5VP', created_at=1702107506, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:daesungsanghoe::8TmSosUv', finished_at=1702108497, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-eOujTwd2bK4aMqI7Oe49Tygg', result_files=['file-ONoalZRFOfBtvZjDFzAgwknN'], status='succeeded', trained_tokens=195639, training_file='file-AtyWWLfHIXPKLCX9SgvZw0eO', validation_file=None)